<a href="https://colab.research.google.com/github/frag30041997/jupyter-one/blob/Cours-Data-Analyst/TP_final_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import seaborn as sns; sns.set
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder as LE
import numpy as np


def accuracy(preds,target):
  return (preds==target).sum()/len(target)


def show_cm(confusion_matrix, labels):
    df_cm = pd.DataFrame(confusion_matrix, labels, labels)
    sns.heatmap(df_cm, annot=True)
    plt.show()

#fonction qui reécrit une colonne pour la refaire en int 
def sci_to_int(feature):
  iplist = []
  for i in range(len(feature)):
    iplist.append(int(feature[i]))
  return iplist

#on part d'un fichier réduit pour raisons pratiques (importation drive impossible sinon)
df_rainb = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/S5_operators.csv',sep=';', encoding="ISO-8859-1",engine='python')
df_rainb.shape

(509692, 12)

#Preprocessing

Le dataset est trop volumineux mais on ne peut pas retirer des données au hasard car on veux des donnés sur plusieurs dates, il va falloir commencer par un gros travail de preprocessing qui définira grandement le rendu par effet papillon.
- version minimale trouvée avec 510 000 lignes

#Phase 2 (transfo data)

In [94]:
df_rainb.head(5)

,platform,dateid,skillrank,role,operator,primaryweapon,secondaryweapon,secondarygadget,nbwins,nbkills,nbdeaths,nbpicks
0,PC,20170210,Bronze,Attacker,BOPE-CAPITAO,M249,PRB92,CLAYMORE,6,6,6,8
1,PC,20170210,Bronze,Attacker,BOPE-CAPITAO,M249,PRB92,STUN GRENADE,4,2,6,8
2,PC,20170210,Bronze,Attacker,BOPE-CAPITAO,PARA-308,PRB92,CLAYMORE,37,40,49,70
3,PC,20170210,Bronze,Attacker,BOPE-CAPITAO,PARA-308,PRB92,STUN GRENADE,17,24,31,41
4,PC,20170210,Bronze,Attacker,G.E.O.-JACKAL,C7E,ITA12S,BREACH CHARGE,74,123,127,174


In [92]:
df_rainb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509692 entries, 0 to 509691
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   platform         509692 non-null  object
 1   dateid           509692 non-null  int64 
 2   skillrank        509692 non-null  object
 3   role             509692 non-null  object
 4   operator         509692 non-null  object
 5   primaryweapon    509692 non-null  object
 6   secondaryweapon  509692 non-null  object
 7   secondarygadget  509692 non-null  object
 8   nbwins           509692 non-null  int64 
 9   nbkills          509692 non-null  int64 
 10  nbdeaths         509692 non-null  int64 
 11  nbpicks          509692 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 46.7+ MB


In [95]:
labenc = LE() #initialisation labelencoder

#uniformisation des datas (+ dictionnaires)
df_rainb['dateid'] = pd.to_datetime(df_rainb['dateid'],format='%Y%m%d')

df_rainb['platform'] = labenc.fit_transform(df_rainb['platform'])
df_rainb['skillrank'] = labenc.fit_transform(df_rainb['skillrank'])
df_rainb['role'] = labenc.fit_transform(df_rainb['role'])
df_rainb['operator'] = labenc.fit_transform(df_rainb['operator'])
df_rainb['primaryweapon'] = labenc.fit_transform(df_rainb['primaryweapon'])
df_rainb['secondaryweapon'] = labenc.fit_transform(df_rainb['secondaryweapon'])
df_rainb['secondarygadget'] = labenc.fit_transform(df_rainb['secondarygadget'])

df_rainb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509692 entries, 0 to 509691
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   platform         509692 non-null  int64         
 1   dateid           509692 non-null  datetime64[ns]
 2   skillrank        509692 non-null  int64         
 3   role             509692 non-null  int64         
 4   operator         509692 non-null  int64         
 5   primaryweapon    509692 non-null  int64         
 6   secondaryweapon  509692 non-null  int64         
 7   secondarygadget  509692 non-null  int64         
 8   nbwins           509692 non-null  int64         
 9   nbkills          509692 non-null  int64         
 10  nbdeaths         509692 non-null  int64         
 11  nbpicks          509692 non-null  int64         
dtypes: datetime64[ns](1), int64(11)
memory usage: 46.7 MB


In [96]:
df_rainb.head(5)

,platform,dateid,skillrank,role,operator,primaryweapon,secondaryweapon,secondarygadget,nbwins,nbkills,nbdeaths,nbpicks
0,0,2017-02-10,0,0,0,21,13,2,6,6,6,8
1,0,2017-02-10,0,0,0,21,13,8,4,2,6,8
2,0,2017-02-10,0,0,0,32,13,2,37,40,49,70
3,0,2017-02-10,0,0,0,32,13,8,17,24,31,41
4,0,2017-02-10,0,0,2,9,4,1,74,123,127,174
